In [25]:
# !pip install xarray
# %pip install netCDF4
# %pip install pyarrow
%pip install -U pandas pyarrow fastparquet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.9 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 33.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Inputs

In [1]:
# import numpy as np
import cdsapi
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Path to your compressed CSV
file_path = "data/ref_catalog_wind_2025.csv.gz"

# Read the CSV file
ref_catalog = pd.read_csv(file_path)
ref_catalog

,bmUnit,quantity,qpos_hours,quantity_raw,n_hours,leadPartyName,bmUnitName,demandCapacity,generationCapacity,gspGroupName,...,power_mws,mean_power,gen_est0,gen_est,lon,lat,era5lon,era5lat,dist_deg,distance_km
0,T_HOWAO-3,1147419.380,6123,1145740.156,6527,Hornsea 1 Limited,HORNSEA_1C,-7.300,400.0,NaN,...,6.997309,4.747981,2.648943e+06,1.797424e+06,1.420740,53.681542,1.50,53.75,0.104732,9.237965
1,T_HOWBO-3,1124547.705,6081,1123143.158,6527,Soundmark Wind Limited,HORNSEA_WF_2C,-5.346,440.0,NaN,...,7.996931,5.406011,2.870778e+06,1.940677e+06,1.539986,53.959998,1.50,54.00,0.056560,5.165377
2,T_HOWAO-2,1112175.105,6096,1110379.289,6527,Hornsea 1 Limited,HORNSEA_1B,-7.260,400.0,NaN,...,6.997309,4.799667,2.648943e+06,1.816991e+06,1.420740,53.681542,1.50,53.75,0.104732,9.237965
3,T_HOWBO-1,1100151.038,6097,1098642.011,6527,Breesea Limited,HORNSEA_WF_2A,-8.741,440.0,NaN,...,7.996931,5.423486,2.870778e+06,1.946950e+06,1.539986,53.959998,1.50,54.00,0.056560,5.165377
4,T_HOWBO-2,1098755.310,6110,1097389.733,6527,Sonningmay Wind Limited,HORNSEA_WF_2B,-5.805,440.0,NaN,...,7.996931,5.362193,2.870778e+06,1.924947e+06,1.539986,53.959998,1.50,54.00,0.056560,5.165377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,T_FALGD-1,0.000,5377,-661.444,6527,Fallago Rig Wind Farm Limited,Fallago Rig Demand,-6.000,0.0,NaN,...,2.388670,1.827647,3.741803e+05,2.862972e+05,-2.688028,55.831308,-2.75,55.75,0.102232,9.847136
212,T_GYMRD-2,0.000,6527,0.000,6527,Gwynt y Mor Offshore Wind Farm,GyM Onshore est Dem 2,-1.000,0.0,NaN,...,3.092194,2.147328,4.036550e+05,2.803122e+05,-3.626649,53.454003,-3.75,53.50,0.131648,9.643765
213,T_GYMRD-E10,0.000,5933,-0.678,6527,Gwynt y Mor Offshore Wind Farm,GyM OSPE non est Dem 10,-1.000,0.0,NaN,...,3.092194,2.119055,4.036550e+05,2.766214e+05,-3.626649,53.454003,-3.75,53.50,0.131648,9.643765
214,T_GYMRD-W10,0.000,6458,-0.074,6527,Gwynt y Mor Offshore Wind Farm,GyM OSPW non est Dem 10,-1.000,0.0,NaN,...,3.092194,2.137876,4.036550e+05,2.790784e+05,-3.626649,53.454003,-3.75,53.50,0.131648,9.643765


## Testing cds api

In [3]:
# Define your location
lon = 0
lat = 0

# Define dataset and request
dataset = "reanalysis-era5-single-levels-timeseries"
request = {
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "100m_u_component_of_wind",
        "100m_v_component_of_wind"
    ],
    "location": {"longitude": lon, "latitude": lat},
    "date": ["2019-01-01/2025-10-25"],
    "data_format": "netcdf"
}

# Create target directory and file name
path = os.path.expanduser("~/Documents/ERA5_at_wf")
os.makedirs(path, exist_ok=True)

target_file = os.path.join(path, f"era5_wind_lon{lon}_lat{lat}.zip")

# Retrieve and download
client = cdsapi.Client()
client.retrieve(dataset, request).download(target_file)


2025-11-05 10:50:32,357 WARNING [2025-03-17T00:00:00] Please be aware that the generation of this dataset is using an alternative source for the ERA5 data and may be subject to changes over time (e.g. file format, data file structure, deprecation etc). This dataset should therefore be regarded as “experimental” and is **not recommended for use in a production environment**. 

Notification of changes via this catalogue entry banner and/or in the [Forum](https://forum.ecmwf.int/) will be provided on best efforts.
2025-11-05 10:50:32,358 INFO Request ID is dc651ef4-71e0-4526-a5f4-9c6684cee278
2025-11-05 10:50:32,434 INFO status has been updated to accepted
2025-11-05 10:50:42,035 INFO status has been updated to running
2025-11-05 10:50:47,164 INFO status has been updated to successful


'/home/s2441782/Documents/ERA5_at_wf/era5_wind_lon0_lat0.zip'

In [4]:
def download_era5_point(lon, lat, path="~/Documents/ERA5_at_wf"):
    """
    Download ERA5 single-level wind data for a specific location (lon, lat)
    and save the zipped file in the given path.
    """
    # Expand and create target directory
    path = os.path.expanduser(path)
    os.makedirs(path, exist_ok=True)

    # Output file name
    target_file = os.path.join(path, f"era5_wind_lon{lon}_lat{lat}.zip")

    # Skip if already downloaded
    if os.path.exists(target_file):
        print(f"✅ File already exists: {target_file}")
        return target_file

    # Define dataset and request
    dataset = "reanalysis-era5-single-levels-timeseries"
    request = {
        "variable": [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "100m_u_component_of_wind",
            "100m_v_component_of_wind"
        ],
        "location": {"longitude": lon, "latitude": lat},
        "date": ["2019-01-01/2025-10-25"],
        "data_format": "netcdf"
    }

    # Retrieve
    client = cdsapi.Client()
    client.retrieve(dataset, request).download(target_file)

    print(f"✅ Download complete: {target_file}")
    return target_file


## Downloading files

In [5]:
# Run downloads in parallel (adjust max_workers as needed)
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [
        executor.submit(download_era5_point, row.lon, row.lat)
        for _, row in ref_catalog.iterrows()
    ]


✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon1.53998641788459_lat53.9599981809586.zip✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon1.42073950284687_lat53.6815416635505.zip
✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon1.42073950284687_lat53.6815416635505.zip
✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon1.53998641788459_lat53.9599981809586.zip

✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon1.42073950284687_lat53.6815416635505.zip
✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon2.17279615900627_lat52.1369621302082.zip
✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon-3.74001064670083_lat54.0899946619529.zip
✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon1.53998641788459_lat53.9599981809586.zip
✅ File already exists: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon-3.74001064670083_lat54

Reading one file

In [6]:
import os
import zipfile
import tempfile
import xarray as xr

In [7]:
# 2. List all zip files
zip_files = [f for f in os.listdir(path) if f.endswith(".zip")]
print(f"Found {len(zip_files)} zip files")

Found 149 zip files


In [8]:
# 3. Pick the first one
zip_path = os.path.join(path, zip_files[0])
print(f"Reading: {zip_path}")

Reading: /home/s2441782/Documents/ERA5_at_wf/era5_wind_lon1.53998641788459_lat53.9599981809586.zip


In [9]:
# Extract the .nc file to a temporary directory and open it
with zipfile.ZipFile(zip_path, "r") as z:
    nc_files = [n for n in z.namelist() if n.endswith(".nc")]
    nc_name = nc_files[0]

    # Extract to a temporary folder
    tmp_dir = tempfile.mkdtemp()
    nc_path = z.extract(nc_name, path=tmp_dir)

    # Open with xarray (using netCDF4 backend)
    ds = xr.open_dataset(nc_path, engine="netcdf4")

print(ds)

<xarray.Dataset> Size: 1MB
Dimensions:     (valid_time: 59760)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 478kB 2019-01-01 ... 2025-10-25T2...
    latitude    float64 8B ...
    longitude   float64 8B ...
Data variables:
    u100        (valid_time) float32 239kB ...
    v100        (valid_time) float32 239kB ...
    u10         (valid_time) float32 239kB ...
    v10         (valid_time) float32 239kB ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2024-09-02T04:48 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts


In [10]:
import pandas as pd

# Convert to pandas DataFrame
df = ds[["u100", "v100", "u10", "v10"]].to_dataframe().reset_index()

# Add static coordinates as columns
df["latitude"] = float(ds.latitude)
df["longitude"] = float(ds.longitude)

# Optional: rename for clarity
df = df.rename(columns={"valid_time": "time"})

# Inspect the result
print(df.head())
print(df.info())


                 time       u100      v100        u10       v10  latitude  \
0 2019-01-01 00:00:00  15.603668  0.433533  11.079956  1.006302      54.0   
1 2019-01-01 01:00:00  15.562622 -0.716660  11.472412  0.153839      54.0   
2 2019-01-01 02:00:00  15.406418 -2.041229  11.772980 -0.945923      54.0   
3 2019-01-01 03:00:00  15.147781 -3.398438  11.509995 -1.995300      54.0   
4 2019-01-01 04:00:00  14.728165 -4.351624  11.196579 -2.773972      54.0   

   longitude  
0        1.5  
1        1.5  
2        1.5  
3        1.5  
4        1.5  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59760 entries, 0 to 59759
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       59760 non-null  datetime64[ns]
 1   u100       59760 non-null  float32       
 2   v100       59760 non-null  float32       
 3   u10        59760 non-null  float32       
 4   v10        59760 non-null  float32       
 5   latit

In [11]:
import os
import zipfile
import tempfile
import xarray as xr
import pandas as pd

def read_era5_zip_to_df(zip_path):
    """Extracts a single .nc from a .zip and returns a tidy DataFrame."""
    with zipfile.ZipFile(zip_path, "r") as z:
        nc_files = [n for n in z.namelist() if n.endswith(".nc")]
        if not nc_files:
            print(f"⚠️ No .nc file in {zip_path}")
            return None

        tmp_dir = tempfile.mkdtemp()
        nc_path = z.extract(nc_files[0], path=tmp_dir)

        # Open dataset
        ds = xr.open_dataset(nc_path, engine="netcdf4")

        # Convert to tidy DataFrame
        df = ds[["u100", "v100", "u10", "v10"]].to_dataframe().reset_index()
        df["latitude"] = float(ds.latitude)
        df["longitude"] = float(ds.longitude)
        df = df.rename(columns={"valid_time": "time"})

    return df


def combine_unique_locations(path="~/Documents/ERA5_at_wf"):
    """Combine all ERA5 .zip files into one DataFrame, skipping repeated locations."""
    path = os.path.expanduser(path)
    zip_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".zip")]

    combined = pd.DataFrame()
    seen_coords = set()

    for zip_path in zip_files:
        try:
            df = read_era5_zip_to_df(zip_path)
            if df is None:
                continue

            coord = (df["latitude"].iloc[0], df["longitude"].iloc[0])
            if coord in seen_coords:
                print(f"⏭️ Skipping duplicate location: {coord}")
                continue

            seen_coords.add(coord)
            combined = pd.concat([combined, df], ignore_index=True)
            print(f"✅ Added {coord} from {os.path.basename(zip_path)}")

        except Exception as e:
            print(f"❌ Error reading {zip_path}: {e}")

    return combined


In [12]:
combined_df = combine_unique_locations("~/Documents/ERA5_at_wf")
print(combined_df.shape)
print(combined_df.head())


✅ Added (np.float64(54.0), np.float64(1.5)) from era5_wind_lon1.53998641788459_lat53.9599981809586.zip
✅ Added (np.float64(53.5), np.float64(-3.25)) from era5_wind_lon-3.18492254214665_lat53.4881830165209.zip
✅ Added (np.float64(57.5), np.float64(-3.25)) from era5_wind_lon-3.36295750893973_lat57.5372637501928.zip
✅ Added (np.float64(56.0), np.float64(1.75)) from era5_wind_lon1.66792096390661_lat55.9673366202541.zip
✅ Added (np.float64(51.75), np.float64(1.25)) from era5_wind_lon1.21801200872639_lat51.7308179434838.zip
✅ Added (np.float64(57.5), np.float64(-3.5)) from era5_wind_lon-3.47620722174696_lat57.4461523015872.zip
✅ Added (np.float64(52.0), np.float64(2.0)) from era5_wind_lon2.03999343559809_lat51.8799917319282.zip
✅ Added (np.float64(55.75), np.float64(-2.75)) from era5_wind_lon-2.83302246285517_lat55.821847948471.zip
✅ Added (np.float64(55.0), np.float64(-4.0)) from era5_wind_lon-4.03211808070007_lat55.119912397996.zip
✅ Added (np.float64(56.25), np.float64(-5.25)) from era5_w

In [13]:
combined_df

,time,u100,v100,u10,v10,latitude,longitude
0,2019-01-01 00:00:00,15.603668,0.433533,11.079956,1.006302,54.0,1.5
1,2019-01-01 01:00:00,15.562622,-0.716660,11.472412,0.153839,54.0,1.5
2,2019-01-01 02:00:00,15.406418,-2.041229,11.772980,-0.945923,54.0,1.5
3,2019-01-01 03:00:00,15.147781,-3.398438,11.509995,-1.995300,54.0,1.5
4,2019-01-01 04:00:00,14.728165,-4.351624,11.196579,-2.773972,54.0,1.5
...,...,...,...,...,...,...,...
5557675,2025-10-25 19:00:00,11.334839,-5.790466,9.717804,-4.714478,53.5,0.5
5557676,2025-10-25 20:00:00,11.672058,-5.483704,10.001297,-4.387665,53.5,0.5
5557677,2025-10-25 21:00:00,11.849121,-5.802872,10.139008,-4.623795,53.5,0.5
5557678,2025-10-25 22:00:00,11.594971,-7.670547,9.988022,-6.214401,53.5,0.5


In [14]:
combined_df.to_parquet(
    "~/Documents/ERA5_at_wf/era5_combined.parquet",
    engine="pyarrow",   # or "fastparquet" if pyarrow fails again
    index=False,
    compression="snappy"
)

